In [12]:

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
root_dir=os.getcwd()
src_path_train = os.path.join(root_dir,"../data/chest_xray/train")
src_path_test = os.path.join(root_dir,"../data/chest_xray/test")
src_path_val = os.path.join(root_dir,"../data/chest_xray/val")
print(os.path.exists(src_path_test))
print(os.path.exists(src_path_train))

train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.20)

test_datagen = ImageDataGenerator(rescale=1 / 255.0)
val_datagen = ImageDataGenerator(rescale=1 / 255.0)

True
True


In [13]:

train_generator = train_datagen.flow_from_directory(
    directory=src_path_train,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    directory=src_path_test,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    directory=src_path_val,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [14]:
def prepare_model_corrected():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(16, activation='relu'))
    # --- Corrected Layer ---
    model.add(Dense(1, activation='sigmoid')) # 1 neuron for binary output
    import tensorflow as tf

# Re-compile the model with a smaller learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) # Default is 0.001
    model.compile(loss="binary_crossentropy",
              optimizer=optimizer,
              metrics=['accuracy'])

    return model

In [15]:
model=prepare_model_corrected()
history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=5)


Epoch 1/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 49s 296ms/step - accuracy: 0.7622 - loss: 0.5401 - val_accuracy: 0.6875 - val_loss: 0.5435
Epoch 2/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 43s 265ms/step - accuracy: 0.8740 - loss: 0.3038 - val_accuracy: 0.7500 - val_loss: 0.4972
Epoch 3/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 43s 266ms/step - accuracy: 0.8970 - loss: 0.2546 - val_accuracy: 0.7500 - val_loss: 0.4524
Epoch 4/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 46s 285ms/step - accuracy: 0.9130 - loss: 0.2197 - val_accuracy: 0.8750 - val_loss: 0.4040
Epoch 5/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 48s 296ms/step - accuracy: 0.9018 - loss: 0.2291 - val_accuracy: 0.8125 - val_loss: 0.4819


In [16]:
import pickle


with open('training_history.pkl', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:

probabilities = model.predict(test_generator)

# 2. Convert probabilities to class labels (0 or 1) using a threshold
# If a probability is > 0.5, it becomes 1 (True), otherwise 0 (False).
y_classes = (probabilities > 0.5).astype("int32")

# Now `y_classes` will correctly contain 0s and 1s based on your model's output
print(y_classes[:10]) # Print the first 10 predicted classes

20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 169ms/step
[[1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]]


In [ ]:
model.save('models/cnn_model.h5')

